In [1]:
%run functions.ipynb

Ready: 2023-04-19 10:06:29


In [2]:
findspark.init()
from pyspark.sql import SparkSession
try:
    spark.stop()
except:
    pass
spark = SparkSession.builder\
        .master("local[*]")\
        .appName('PySpark_Tutorial')\
        .getOrCreate()
spark

In [28]:
request_df = spark.read.parquet(f'files/request_df')
request_df.createOrReplaceTempView('request_df')

In [35]:
requests_filtered_df = spark.sql('''

with cte as 
    (
    select
        rubricID
        ,sum(cnt_requests) cnt_requests
    from request_df
    group by rubricID
    )
, cte2 as 
    (
    select *, row_number() over(partition by rubricID order by cnt_requests desc) rn
    from cte
    )

select 
    f.rubricName
    ,f.time
    ,f.date
    ,f.tx
from final_df as f
join cte2 as c
    on c.rubricID = f.rubricID and rn < 101
    
    
    ''')

requests_filtered_df.write.mode('overwrite').parquet(f'files/requests_filtered_df')

In [36]:
import string
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")

def normalize_word(word):
    return (sorted([morph.normal_forms(str(i))[0].strip(string.punctuation + string.whitespace) for i in word.split(' ')]))

normalize_word_udf = udf(normalize_word, ArrayType(StringType()))

def check_stopwords(word, russian_stopwords = russian_stopwords):
    if word not in russian_stopwords and len(word)>1:
        for letter in word:
            if letter in string.punctuation or letter in string.digits:
                return 0
        return 1
    return 0

check_stopwords_udf = udf(check_stopwords, StringType())

In [4]:
name = 'ten_city'

In [39]:
df = spark.read.parquet(f'files/requests_filtered_df')

df.createOrReplaceTempView('df')

grouped_df = spark.sql('''

select
    rubricName
    ,year
    ,tx
    ,cnt
    ,row_number() over(partition by rubricName ,year order by cnt desc) rn
from (
    select
        rubricName
        ,year(date) year
        ,lower(tx) tx
        ,count(*) cnt
    from df
    group by 
        rubricName
        ,year(date)
        ,lower(tx)
        ) t
''').where('tx is not null and year in (2020,2021,2022,2023)')#.where('rn < 16')


grouped_df.write.mode('overwrite').parquet(f'files/df_tx_grouped_{name}')

In [78]:
(spark.read.parquet(f"files/df_tx_grouped_{name}")
 .where('tx is not null and year in (2020,2021,2022,2023)')
 .write.partitionBy('rubricName').mode('overwrite')
 .parquet(f'files/df_tx_grouped_filtered_{name}')
)

df_tx_grouped_filtered = spark.read.parquet(f"files/df_tx_grouped_filtered_{name}")
df_tx_grouped_filtered.createOrReplaceTempView('df_tx_grouped_filtered')
df_tx_grouped_filtered.where('1=0').write.mode('overwrite').parquet(f'files/df_tx_grouped_normalize_{name}')

rubrics = spark.sql('''select distinct rubricName from df_tx_grouped_filtered''').collect()
rubrics = [i.asDict()['rubricName'] for i in rubrics]

In [96]:
for rubric in tqdm(rubrics):
    df_i = df_tx_grouped_filtered.where(f'rubricName="{rubric}"')
    df_i_norm = df_i.withColumn('new_tx',normalize_word_udf(col('tx')))
    df_i_norm.write.mode('append').parquet(f'files/df_tx_grouped_normalize_{name}')

 92%|██████████████████████████████████████████████████████████████████████▏     | 1541/1669 [4:32:03<22:35, 10.59s/it]


ParseException: 
Syntax error at or near 'М'(line 1, pos 13)

== SQL ==
rubricName=""Металлопрокат и металлоизделия от компании ""Металлтрейд-Новосибирск""""
-------------^^^


In [105]:
for rubric in tqdm(rubrics[1541:]):
    df_i = df_tx_grouped_filtered.where(f'''rubricName='{rubric}' ''')
    df_i_norm = df_i.withColumn('new_tx',normalize_word_udf(col('tx')))
    df_i_norm.write.mode('append').parquet(f'files/df_tx_grouped_normalize_{name}')

100%|████████████████████████████████████████████████████████████████████████████████| 128/128 [03:34<00:00,  1.67s/it]


In [20]:
df = pd.read_excel("D:\файлы_2гис_2023\Калининград\Мастер_таблица_2ГИС_Калининград_230412_2300.xlsx").fillna(0)

df = df[df['Рубрика'] != 'Итог']

df[df.columns[1:]] = df[df.columns[1:]].astype('int64')

df['Запросы, итого'] = df['Запросы 2020'] + df['Запросы 2021'] + df['Запросы 2022'] + df['Запросы 2023']

df['Запросов на пользователя 2020'] = df['Запросы 2020'] / df['Пользователи 2020']
df['Запросов на пользователя 2021'] = df['Запросы 2021'] / df['Пользователи 2021']
df['Запросов на пользователя 2022'] = df['Запросы 2022'] / df['Пользователи 2022']
df['Запросов на пользователя 2023'] = df['Запросы 2023'] / df['Пользователи 2023']
df['Запросов на пользователя, итого'] = df['Запросы, итого'] / df['Пользователи, итого']

df['Пользователи, %'] = df['Пользователи, итого'] / 524647 *100
df['Запросы, %'] = df['Запросы, итого'] / df['Запросы, итого'].sum() *100
df['№'] = df.index

df = df[['№', 'Рубрика', 'Пользователи 2020', 'Пользователи 2021',
       'Пользователи 2022', 'Пользователи 2023', 'Пользователи, итого',
       'Пользователи, %', 'Запросы 2020', 'Запросы 2021', 'Запросы 2022',
       'Запросы 2023', 'Запросы, итого', 'Запросы, %',
       'Запросов на пользователя 2020', 'Запросов на пользователя 2021',
       'Запросов на пользователя 2022', 'Запросов на пользователя 2023',
       'Запросов на пользователя, итого']]

d = {'2020': 178579,
 '2021': 184407,
 '2022': 213106,
 '2023': 203121,
 'Итого': 524647}

In [21]:
x = count_ratings(df, 100)

len(x.columns)

Рейтинги построены


50

In [23]:
new_df1 = optimize_df(df, index=60, cut_level=1300)

(1180, 69)
Рейтинги построены
Горизонтальная оптимизация


100%|██████████████████████████████████████████████████████████████████████████████| 1180/1180 [01:21<00:00, 14.47it/s]


Вертикальная оптимизация


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [03:13<00:00, 19.34s/it]


,Оригинальный файл,Горизонтальная оптимизация,Вертикальная оптимизация
cnt_horizontal,,,
0.0,544,544,544
1.0,213,564,373
2.0,99,18,178
3.0,59,7,83
4.0,26,9,1
5.0,22,8,1
6.0,33,3,0
7.0,35,4,0
8.0,33,6,0


In [24]:
final_df = make_final(df, new_df1)

C:\Users\Mamed\AppData\Local\Temp\ipykernel_8240\982461796.py:411: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  final_svod['Рейтинг суммарный'] = final_svod[optimized_svod.columns].sum(axis=1)
C:\Users\Mamed\AppData\Local\Temp\ipykernel_8240\982461796.py:412: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  final_svod['Рейтинг средний'] = final_svod[optimized_svod.columns].mean(axis=1)


In [25]:
final_df.to_excel('Мастер_таблица_2ГИС_Калининград_230412_2330.xlsx', index=False)

In [26]:
df = pd.read_excel("Мастер_таблица_2ГИС_Калининград_230412_2330.xlsx")

In [27]:
df['Запросов на пользователя,\nитого'] = df['Запросов на пользователя, итого']
#df['Рейтинг по пользователям,\nитого'] = df['Рейтинг по пользователям, итого']
#df['Рейтинг запросов\nна пользователя 2020'] = df['Рейтинг запросов на пользователя 2020']
#df['Рейтинг запросов\nна пользователя 2021'] = df['Рейтинг запросов на пользователя 2021']
#df['Рейтинг запросов\nна пользователя 2022'] = df['Рейтинг запросов на пользователя 2022']
#df['Рейтинг запросов\nна пользователя 2023'] = df['Рейтинг запросов на пользователя 2023']
#df['Рейтинг запросов\nна пользователя, итого'] = df['Рейтинг запросов на пользователя, итого']

df = df.drop(

['Запросов на пользователя, итого'
#,'Рейтинг по пользователям, итого'
#,'Рейтинг запросов на пользователя 2020'
#,'Рейтинг запросов на пользователя 2021'
#,'Рейтинг запросов на пользователя 2022'
#,'Рейтинг запросов на пользователя 2023'
#,'Рейтинг запросов на пользователя, итого'
]
,axis=1
)

In [28]:
df.to_excel("Мастер_таблица_2ГИС_Калининград_230412_2345.xlsx", index=False)

In [ ]:
make_ratings_word("Мастер_таблица_2ГИС_Калининград_230412_2345.xlsx", header_color='#B76C00', row_color='#FFF4D9')

In [41]:
df = pd.read_excel('files/kaliningrad/requests_text_kaliningrad_230415_1101.xlsx')

In [51]:
add_links('Заготовка_2ГИС_Калининград_нулевая_рубрика_230417_2315.docx', '', df[df['Рейтинг рубрики'] == 0]['Поисковый запрос'].values, 'Калининград')

In [49]:
df = pd.read_excel('files/astana/requests_text_astana_230409_1527.xlsx')

In [52]:
add_links('Заготовка_2ГИС_Астана_нулевая_рубрика_230417_2315.docx', '', df[df['Рейтинг рубрики'] == 0]['Поисковый запрос'].values, 'Астана')